In [114]:
# 加载pyecharts
import os
import re
import pandas
from os import walk
from pyecharts import Geo, Bar, Line, Overlap, EffectScatter, Map, Style, Page

In [37]:
# 城市 -- 指定省的城市 xx市
city = ['延庆区', '怀柔区', '昌平区', '海淀区', '朝阳区', '顺义区', '平谷区', '密云区',
        '大兴区', '西城区', '东城区', '门头沟区', '房山区', '通州区', '丰台区', '石景山区']
values2 = [1.07, 3.85, 6.38, 8.21, 2.53, 4.37, 9.38,
           4.29, 6.1, 1.07, 3.85, 6.38, 8.21, 2.53, 4.37, 2.53]


# 河南地图  数据必须是省内放入城市名
map2 = Map("北京地图", '北京', width=1400, height=800)
map2.add('北京', city, values2, visual_range=[1, 10],
         maptype='北京', is_visualmap=True, visual_text_color='#000', is_map_symbol_show=False)
map2

In [124]:
def visualize(df, name):
    # 导入自定义的地点经纬度
    geo_cities_coords = {df.iloc[i]['poi_name']: [df.iloc[i]['lng'], df.iloc[i]['lat']]
                         for i in range(len(df))}  # 根据文件大小生成字典
    attr = list(df['poi_name'])  # 字典的每个键值
    value = list(df['poi_porb'])  # 由于量值的太大，换算以下(散点的颜色就是和这个想关的)
    style = Style(title_color="#fff", title_pos="center",
                  width=1200, height=600, background_color="#404a59")

    # 可视化
    geo = Geo(name, **style.init_style)
    geo.add("", attr, value, visual_range=[0.2, 1], symbol_size=10,
            visual_text_color="#fff", is_piecewise=True,
            is_visualmap=True, maptype='北京', visual_split_number=10,
            geo_cities_coords=geo_cities_coords)
    return geo

In [125]:
def prepare(dirs):
    file_path_list = []
    file_name_list = []
    dataframe_list = []
    # walk会返回3个参数，分别是路径，目录list，文件list，你可以按需修改下
    for root, _, files in walk(dirs):
        for file in files:
            if '.csv' in file:
                file_path = root + "\\" + file
                file_path_list.append(file_path)
                file_name = re.split(r'[\\.]\s*', file_path)
                file_name = ''.join(filter(lambda s: isinstance(s, str) and len(
                    s) <= 5 and s != "" and s != "data" and s != "csv", file_name))
                file_name_list.append(file_name)
                
                dataframe = pandas.read_csv(file_path)  # 读取文件
                dataframe_list.append(dataframe)
                
    return file_name_list, dataframe_list


In [128]:
dirs = ".\\2019-04-14-17-01-26-visualization\\data"

file_name_list, dataframe_list = prepare(dirs)

if len(file_name_list) != len(dataframe_list):
    raise Exception("数量不匹配，请检查代码")

page = Page()
for i in range(len(file_name_list)):
    geo = visualize(dataframe_list[i], file_name_list[i])
    page.add(geo)
page.render("Over All Visualization.html")